# Gain Statistical Insights into Your Data

Woodwork provides methods on your DataFrames to allow you to use the typing information stored by Woodwork to better understand your data.

Follow along to learn how to use Woodwork's statistical methods on a DataFrame of retail data while demonstrating the full capabilities of the functions.

In [ ]:
import pandas as pd
from woodwork.demo import load_retail

df = load_retail()
df.ww

## DataFrame.ww.describe

Use `df.ww.describe()` to calculate statistics for the columns in a DataFrame, returning the results in the format of a pandas DataFrame with the relevant calculations done for each column.

In [ ]:
df.ww.describe()

There are a couple things to note in the above dataframe:

- The Woodwork index, `order_product_id`, is not included
- We provide each column's typing information according to Woodwork's typing system
- Any statistics that can't be calculated for a column, such as `num_false` on a `Datetime` are filled with `NaN`.
- Null values do not get counted in any of the calculations other than `nunique`

## DataFrame.ww.value_counts

Use `df.ww.value_counts()` to calculate the most frequent values for each column that has `category` as a standard tag. This returns a dictionary where each column is associated with a sorted list of dictionaries. Each dictionary contains `value` and `count`.

In [ ]:
df.ww.value_counts()

## DataFrame.ww.mutual_information

`df.ww.mutual_information` calculates the mutual information between all pairs of relevant columns. Certain types, like strings, can't have mutual information calculated.

The mutual information between columns `A` and `B` can be understood as the amount of knowledge you can have about column `A` if you have the values of column `B`. The more mutual information there is between `A` and `B`, the less uncertainty there is in `A` knowing `B`, and vice versa. 

In [ ]:
df.ww.mutual_information()

#### Available Parameters
`df.ww.mutual_information` provides various parameters for tuning the mutual information calculation.

- `num_bins` - In order to calculate mutual information on continuous data, Woodwork bins numeric data into categories. This parameter allows you to choose the number of bins with which to categorize data.
    - Defaults to using 10 bins
    - The more bins there are, the more variety a column will have. The number of bins used should accurately portray the spread of the data.
- `nrows` - If `nrows` is set at a value below the number of rows in the DataFrame, that number of rows is randomly sampled from the underlying data
    - Defaults to using all the available rows.
    - Decreasing the number of rows can speed up the mutual information calculation on a DataFrame with many rows, but you should be careful that the number being sampled is large enough to accurately portray the data.
- `include_index` - If set to `True` and an index is defined with a logical type that is valid for mutual information, the index column will be included in the mutual information output.
    - Defaults to `False`

Now that you understand the parameters, you can explore changing the number of bins. Note—this only affects numeric columns `quantity` and `unit_price`. Increase the number of bins from 10 to 50, only showing the impacted columns.

In [ ]:
mi = df.ww. mutual_information()
mi[mi['column_1'].isin(['unit_price', 'quantity']) | mi['column_2'].isin(['unit_price', 'quantity'])]

In [ ]:
mi = df.ww.mutual_information(num_bins = 50)
mi[mi['column_1'].isin(['unit_price', 'quantity']) | mi['column_2'].isin(['unit_price', 'quantity'])]

In order to include the index column in the mutual information output, run the calculation with `include_index=True`.

In [ ]:
mi = df.ww.mutual_information(include_index=True)
mi[mi['column_1'].isin(['order_product_id']) | mi['column_2'].isin(['order_product_id'])]